<a href="https://colab.research.google.com/github/chaves08r/House-price/blob/main/House_Price1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports <a id="imports"></a>

In [72]:
import os
os.environ['KAGGLE_USERNAME'] = "jpso0907"
os.environ['KAGGLE_KEY'] = "3ec703ca5503f4f9e20de4fe23482a8b"

!kaggle competitions download -c home-data-for-ml-course

home-data-for-ml-course.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip home-data-for-ml-course.zip

Archive:  home-data-for-ml-course.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Visualizar os dados
train_data.head()

In [ ]:
# Importa a biblioteca numpy, usada para manipulação de arrays e operações numéricas.
import numpy as np

# Importa a biblioteca pyplot do matplotlib, usada para criar gráficos.
from matplotlib import pyplot as plt

# Importa a biblioteca seaborn, que é uma extensão do matplotlib para criar gráficos mais atraentes e informativos.
import seaborn as sns

# Função para dividir os dados em treino, validação e teste
from sklearn.model_selection import train_test_split


# Importa a biblioteca de warnings para controlar a exibição de mensagens de aviso.
import warnings

# Ignora todas as mensagens de aviso, evitando que apareçam no terminal durante a execução do código.
warnings.filterwarnings("ignore")


## Dados gerais

O conjunto de dados apresenta 81 variáveis explicativas que detalham quase todos os aspectos de imóveis residenciais localizados em Ames, Iowa. Essas variáveis abrangem características estruturais, qualidade de construção, localização, comodidades e condições de venda das propriedades. O objetivo da análise é prever o preço final de venda de cada imóvel (SalePrice), com base nesses atributos.

acesse os dados aqui: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview

In [ ]:
# Criação do dicionário simplificado com dupla descrição (breve e detalhada)
dictionary = {
    "SalePrice": ("Preço de venda do imóvel.", "Valor de venda em dólares — variável alvo da predição."),
    "MSSubClass": ("Classe do edifício.", "Identifica o tipo de habitação envolvida na venda."),
    "MSZoning": ("Zoneamento do imóvel.", "Classificação geral de zoneamento urbano."),
    "LotFrontage": ("Frente do lote.", "Metros lineares de rua conectados à propriedade."),
    "LotArea": ("Área do lote.", "Tamanho total do terreno em pés quadrados."),
    "Street": ("Tipo de rua.", "Tipo de acesso à via pública."),
    "Alley": ("Tipo de viela.", "Tipo de acesso por viela — se houver."),
    "LotShape": ("Formato do lote.", "Forma geral do terreno (regular, irregular, etc.)."),
    "LandContour": ("Topografia do terreno.", "Nível de planicidade do terreno."),
    "Utilities": ("Infraestrutura básica.", "Tipo de utilidades disponíveis (água, eletricidade, etc.).")
}

# Criação do DataFrame
dict_df = pd.DataFrame(dictionary.items(), columns=["Variável", "Info"])

# Separar descrição e categoria
dict_df["Descrição"] = dict_df["Info"].apply(lambda x: x[0])
dict_df["Categorias"] = dict_df["Info"].apply(lambda x: x[1])

# Determinar o tipo de variável
numericas = ["SalePrice", "LotFrontage", "LotArea"]
dict_df["Tipo"] = dict_df["Variável"].apply(lambda x: "Numérica" if x in numericas else "Categórica")

# Remover coluna auxiliar
dict_df.drop(columns=["Info"], inplace=True)


In [ ]:
display(dict_df)

In [ ]:
# Lista de nomes das colunas
columns = ["LotArea",
"GrLivArea",
"FullBath",
"HalfBath",
"YearBuilt",
"YearRemodAdd",
"OverallQual",
"OverallCond",
"KitchenQual",
"BsmtQual",
"Neighborhood",
"MSZoning",
"Condition1",
"SaleType",
"SaleCondition",
"YrSold",
"MoSold",
"SalePrice"
]

# Ler os dados a partir do arquivo train.csv
df = pd.read_csv('train.csv', usecols=columns)

# Exibir as 3 primeiras linhas do DataFrame para verificar a estrutura dos dados
display(df.head(3))

In [ ]:
#informações gerais da base
train_data.info()

## Tratamento

In [ ]:
#retornando o shape da base
train_data.shape

In [ ]:
# verifica linhas duplicadas

df.duplicated().sum()

In [ ]:
#visualizando a quantidade valores vazios, listando as colunas e as 20 primeiras linhas
train_data.isnull().sum().sort_values(ascending=False).head(20)

In [ ]:
#percentual de valores vazios
(train_data.isnull().sum() / train_data.shape[0] * 100).sort_values(ascending=False).head(20)

In [ ]:
#eliminando colunas com + de 10% de valores vazios
eliminar = train_data.columns[(train_data.isnull().sum() / train_data.shape[0]) > 0.1]

eliminar

In [ ]:
train_data = train_data.drop(eliminar, axis=1)

Nesse primeiro modelo foi eliminado as colunas de texto, tratou-se os valores vazios e utilizou-se alguns algoritmos para testar e avaliar possiveis erros.

In [ ]:
(train_data.isnull().sum() / train_data.shape[0] * 100).sort_values(ascending=False).head(20)

In [ ]:
#colunas com valores númericos
colunas = train_data.columns[train_data.dtypes != 'object']
colunas

In [ ]:
train_data2 = train_data.loc[:,colunas]
train_data2.head()

In [ ]:
train_data2.isnull().sum().sort_values(ascending=False).head(3)

In [ ]:
#substituindo valores vazios por -1
train_data2 = train_data2.fillna(-1)

In [ ]:
train_data2

Modelos

separação de base de treino e teste:
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
# determinação de x e y

x = train_data2.drop('SalePrice', axis=1)
y = train_data2.SalePrice

In [ ]:
#import train test split
from sklearn.model_selection import train_test_split

In [ ]:
#separação da base de treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Algoritmos utilizados:

Regressão Linear -
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html



*   A regressão linear é um modelo matemático simples e poderoso usado para prever um valor com base em outro. Em outras palavras, ela tenta encontrar uma linha reta que melhor se ajusta aos dados e mostra como uma variável influencia outra



Árvore de Regressão -
https://scikit-learn.org/stable/modules/tree.html#regression

* Uma árvore de regressão é um modelo de machine learning que toma decisões em forma de “árvore” para prever valores numéricos (como o preço de uma casa). Ela funciona dividindo os dados em grupos menores e mais homogêneos, passo a passo, até chegar a uma previsão.

KNeighborsRegressor -
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor

* O KNeighborsRegressor é um modelo de machine learning baseado em vizinhos mais próximos usado para fazer previsões numéricas (regressão). Ele é uma versão do algoritmo KNN (K-Nearest Neighbors) aplicada à regressão.

In [ ]:
#importando a regressão linear
from sklearn.linear_model import LinearRegression

In [ ]:
# Criando o regressor e fazendo o fit com os dados de treinoreg_rl = LinearRegression().fit(x_train, y_train)
reg_rl = LinearRegression().fit(x_train, y_train)


In [ ]:
#previsão comos dados de teste
y_rl = reg_rl.predict(x_test)

In [ ]:
#importando arvore de regressão
from sklearn import tree


In [ ]:
# criando o regressor e fazendo fit com os dados de treino
reg_arvore = tree.DecisionTreeRegressor(random_state=42).fit(x_train, y_train)

In [ ]:
#previsão
y_ar = reg_arvore.predict(x_test)

In [ ]:
#importando o KNN
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# Criando o regressor e fazendo o fit com os dados de treino
reg_knn = KNeighborsRegressor(n_neighbors=2).fit(x_train, y_train)

In [ ]:
#previsão
y_knn = reg_knn.predict(x_test)

Utilizando tanto o erro absoluto quanto o quadrático:
Erro médio absoluto -
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html

* O Erro Médio Absoluto (ou MAE – Mean Absolute Error, em inglês) é uma métrica usada para medir o quão erradas estão as previsões de um modelo em relação aos valores reais. Ele calcula a média das diferenças absolutas entre os valores previstos e os valores reais.

Erro quadrático médio -
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

* O EQM mede a diferença entre os valores reais e os valores previstos por um modelo. Ele calcula o quadrado dessa diferença (para não cancelar erros positivos e negativos) e tira a média de todos esses quadrados.

In [ ]:
#importando o erro médio absoluto

from sklearn.metrics import mean_absolute_error

In [ ]:
#import erro quadratico médio
from sklearn.metrics import mean_squared_error

In [ ]:
#avaliando erro na regressão
print(mean_absolute_error(y_test, y_rl))
print(mean_squared_error(y_test, y_rl))

In [ ]:
#avaliando arvore de decisao
print(mean_absolute_error(y_test, y_ar))
print(mean_squared_error(y_test, y_ar))

In [ ]:
#avaliando knn
print(mean_absolute_error(y_test, y_knn))
print(mean_squared_error(y_test, y_knn))

plotando visualmente a relação do y_test com as previsões feitas


In [ ]:
#Criação  do gráfico
fig, ax = plt.subplots(ncols=3, figsize=(15, 5))

ax[0].scatter(y_test/100000,y_rl/100000)
ax[0].plot([0,700000], [0,700000], '--r')
ax[1].scatter(y_test/100000,y_ar/100000)
ax[1].plot([0,700000], [0,700000], '--r')
ax[2].scatter(y_test/100000,y_knn/100000)
ax[2].plot([0,700000], [0,700000], '--r')

ax[0].set(xlim=(0, 7), ylim=(0, 7))
ax[0].set_xlabel('Real')
ax[0].set_ylabel('Previsto')
ax[1].set(xlim=(0, 7), ylim=(0, 7))
ax[1].set_xlabel('Real')
ax[1].set_ylabel('Previsto')
ax[2].set(xlim=(0, 7), ylim=(0, 7))
ax[2].set_xlabel('Real')
ax[2].set_ylabel('Previsto')

A Regressão Linear foi escolhida como modelo final por apresentar o menor erro quadrático médio (EQM) entre os algoritmos testados, métrica essa que também é adotada pelo Kaggle para avaliar e classificar modelos em competições de regressão. O EQM penaliza com mais intensidade os erros maiores, sendo ideal para priorizar previsões mais precisas. Visualmente, essa superioridade é confirmada pela maior concentração dos pontos próximos à linha de referência (valores reais = valores previstos) nos gráficos de dispersão, indicando um bom ajuste aos dados. Além disso, a simplicidade e interpretabilidade da Regressão Linear tornam o modelo mais transparente e fácil de comunicar, especialmente em contextos que exigem clareza na tomada de decisão.


## Fazendo a previsão para a base de teste da competição


retirou-se os valores vazios pois a regressão linear não vai conseguir trabalhar com valores vazio
Se tentarmos usar essa base, ele vai retornar um erro dizendo que o input possui valores NaN
Podemos apenas substituir por -1 como fizemos acima

In [ ]:
#importando base de teste
teste = pd.read_csv('test.csv')

In [ ]:
teste.head(3)

In [ ]:
#limapndo colunas da base de treino
teste = teste.drop(eliminar, axis =1)

In [ ]:
 #verificando colunas numericas
coluna2 = teste.columns[teste.dtypes != 'object']
coluna2

In [ ]:
#mantendo apenas colunas numericas
teste = teste.loc[:,coluna2]

In [ ]:
teste.info()

In [ ]:
#visualizando valores vazios
teste.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
#substituindo valores vazios por -1
teste = teste.fillna(-1)

In [ ]:
#previsao - regressao linear
y_pred = reg_rl.predict(teste)

In [ ]:
#adicionando a coluna de previsao na base
teste['SalePrice'] = y_pred

In [ ]:
#extração de id e saleprice
resultado = teste[['Id','SalePrice']]
resultado.head(3)

In [ ]:
resultado.to_csv('submission.csv', index=False)

## Visualizações

In [ ]:
#Quanto maior a área construída acima do solo, maior o preço de venda?
sns.scatterplot(x='GrLivArea', y='SalePrice', data=train_data)
plt.title('Área construída vs Preço de venda')


O gráfico acima mostra uma relação positiva entre o tamanho das casas (medido por `GrLivArea`) e seus respectivos preços (`SalePrice`). Observa-se que, de forma geral, quanto maior a área construída, maior tende a ser o valor de venda, indicando que o tamanho do imóvel é um fator importante na precificação. A maior concentração de pontos se dá entre 1.000 e 2.500 pés quadrados e entre 100.000 e 300.000 dólares, refletindo o perfil mais comum das casas analisadas. Apesar disso, existem alguns outliers, casas muito grandes ou muito caras, que se destacam e podem influenciar a performance de modelos preditivos, sendo recomendável analisá-los separadamente.

In [ ]:
#Existe uma relação linear positiva entre o tamanho da casa e o preço.
sns.scatterplot(x='GrLivArea', y='SalePrice', data=df)
plt.title("Área construída vs Preço de venda (com linha de tendência)")
sns.regplot(x='GrLivArea', y='SalePrice', data=df, scatter=False, color='red')


A linha de tendência, quando adicionada ao gráfico de dispersão entre GrLivArea (área construída) e SalePrice (preço de venda), representa a melhor reta que resume a relação entre as duas variáveis. Ela indica a direção geral dos dados e permite visualizar de forma clara que, conforme a área construída aumenta, o preço de venda também tende a crescer. Essa linha é calculada com base em regressão linear e ajuda a identificar padrões, facilitando a previsão de preços. Além disso, ela permite observar se os pontos estão bem distribuídos ao redor dela — o que indica um bom ajuste — ou se há desvios significativos (resíduos), sugerindo a influência de outras variáveis ou a presença de outliers. Em resumo, a linha de tendência serve como um guia visual para entender a força e o tipo de relação entre área e preço no conjunto de dados.

In [ ]:
#Casas em determinados bairros têm preços sistematicamente mais altos
plt.figure(figsize=(15,6))
sns.boxplot(x='Neighborhood', y='SalePrice', data=train_data)
plt.xticks(rotation=45)
plt.title('Preço por Vizinhança')


O gráfico apresenta a variação dos preços de venda (SalePrice) das casas em diferentes bairros (Neighborhood) e evidencia como a localização influencia significativamente o valor dos imóveis. Bairros como NoRidge, NridgHt e StoneBr se destacam por possuírem preços medianos mais altos, indicando áreas mais valorizadas, enquanto regiões como IDOTRR e BrDale concentram preços mais baixos. A altura das caixas revela a dispersão dos valores dentro de cada bairro, e os pontos fora delas representam outliers — casas com preços muito acima ou abaixo da média local. Essa visualização destaca que o bairro não apenas afeta o preço médio, mas também a consistência dos valores, sendo uma variável essencial na modelagem preditiva do preço de imóveis.

In [ ]:
#Casas com melhor qualidade de construção valem mais?
sns.violinplot(x='OverallQual', y='SalePrice', data=train_data)
plt.title('Qualidade Geral vs Preço de Venda')


O gráfico mostra a relação entre a qualidade geral dos imóveis (OverallQual) e seus preços de venda (SalePrice), revelando uma forte correlação positiva entre essas variáveis. À medida que a nota de qualidade aumenta, observa-se também um aumento consistente no valor dos imóveis, indicando que casas mais bem avaliadas tendem a ser significativamente mais caras. As faixas de qualidade mais baixas (de 1 a 4) concentram-se em preços menores e mais uniformes, enquanto as notas mais altas (principalmente de 8 a 10) apresentam não apenas preços elevados, mas também maior variação, refletindo imóveis de alto padrão com características variadas. A visualização reforça a importância da qualidade geral como um dos principais fatores na definição do valor de mercado de uma casa.

In [ ]:
#Casas mais novas tendem a ser mais caras
sns.scatterplot(x='YearBuilt', y='SalePrice', data=train_data)
plt.title('Ano de Construção vs Preço de Venda')


O gráfico mostra que há uma relação clara entre o ano de construção dos imóveis (`YearBuilt`) e seus preços de venda (`SalePrice`), indicando que casas mais recentes tendem a ser vendidas por valores mais altos. Observa-se uma concentração de construções após 1950, período em que os preços começam a subir gradualmente, com um aumento mais expressivo a partir dos anos 2000. Imóveis muito antigos, especialmente os construídos antes de 1940, apresentam preços mais baixos e pouca variação, enquanto os mais novos mostram maior dispersão e alguns valores elevados, refletindo construções modernas e de padrão superior. Esses padrões sugerem que o ano de construção é uma variável relevante na previsão do preço de venda.


In [ ]:
#Mais banheiros significam casas mais valorizadas?
sns.barplot(x='FullBath', y='SalePrice', data=train_data)
plt.title('Número de Banheiros Completos vs Preço')


O gráfico de barras mostra que o número de banheiros completos (`FullBath`) está diretamente relacionado ao preço médio de venda dos imóveis (`SalePrice`). De maneira geral, quanto maior a quantidade de banheiros completos, maior tende a ser o valor da casa, com destaque para imóveis com três banheiros, que apresentam os preços médios mais altos. Casas com dois banheiros também se destacam por estarem acima da média geral, enquanto aquelas com apenas um banheiro têm os menores valores médios. Curiosamente, imóveis com zero banheiros completos possuem preços médios superiores aos de um banheiro, o que pode indicar a presença de banheiros parciais ou outras características específicas nesses imóveis. A tendência crescente reforça que a quantidade de banheiros é um importante indicador de conforto e valorização do imóvel.


In [ ]:
#Casas com garagens maiores são mais caras
sns.boxplot(x='GarageCars', y='SalePrice', data=train_data)
plt.title('Capacidade da Garagem vs Preço')


O gráfico mostra que a capacidade da garagem (`GarageCars`) está relacionada ao preço de venda dos imóveis (`SalePrice`), evidenciando que casas com garagens maiores tendem a ter valores mais altos. Imóveis com capacidade para três carros se destacam com os maiores preços medianos e ampla variação, indicando um padrão de construção mais elevado. Casas com uma ou nenhuma vaga de garagem concentram os preços mais baixos, enquanto aquelas com duas vagas apresentam valores medianos intermediários e mais frequentes. Curiosamente, imóveis com quatro vagas não seguem totalmente essa tendência, com mediana mais baixa, possivelmente devido à baixa quantidade de observações ou influência de outros fatores. De forma geral, a presença e o tamanho da garagem funcionam como um bom indicador do nível de valorização do imóvel.


In [ ]:
#sns.histplot(df['SalePrice'], kde=True)

sns.histplot(df['SalePrice'], kde=True)


O gráfico de histograma com curva de densidade mostra a distribuição dos preços de venda dos imóveis (`SalePrice`) no conjunto de dados, revelando uma concentração significativa de casas com valores entre 100.000 e 200.000 dólares. A distribuição é assimétrica à direita, com uma cauda longa indicando a presença de imóveis com preços elevados, embora menos frequentes. Esse comportamento sugere que a maioria dos imóveis possui preços acessíveis, enquanto apenas uma pequena parcela corresponde a casas de alto padrão. Essa assimetria é comum em dados financeiros e indica que, para análises estatísticas mais precisas, pode ser vantajoso aplicar transformações como o logaritmo para aproximar a distribuição de uma curva normal.


In [ ]:
#Casas com maior qualidade de construção têm preços significativamente mais altos.
sns.boxplot(x='OverallQual', y='SalePrice', data=df)


O gráfico evidencia uma relação positiva entre a qualidade geral dos imóveis (`OverallQual`) e seus preços de venda (`SalePrice`), demonstrando que quanto maior a nota de qualidade, maior tende a ser o valor do imóvel. Observa-se um crescimento progressivo nos preços medianos à medida que a qualidade aumenta, com destaque para as casas avaliadas com notas 9 e 10, que apresentam valores significativamente mais altos e grande dispersão. Essa variabilidade nos níveis superiores sugere a presença de imóveis de luxo com características diferenciadas. Já os imóveis com notas entre 5 e 6 concentram a maior parte das observações, indicando que a maioria das casas analisadas possui qualidade intermediária. Essa visualização reforça a importância da qualidade da construção como um dos principais fatores de valorização no mercado imobiliário.


In [ ]:
#Algumas variáveis estão altamente correlacionadas com SalePrice.
sns.heatmap(df.corr(numeric_only=True), annot=True, fmt=".2f", cmap="coolwarm")


O mapa de calor apresenta a correlação entre variáveis numéricas do conjunto de dados, destacando aquelas que mais influenciam o preço de venda dos imóveis (`SalePrice`). As variáveis com maior correlação positiva são `OverallQual` (qualidade geral da construção), `GrLivArea` (área útil) e `YearBuilt` (ano de construção), indicando que casas maiores, mais novas e de melhor qualidade tendem a ser mais valorizadas. Já variáveis como `MoSold` (mês de venda) e `YrSold` (ano de venda) apresentam correlação próxima de zero, sugerindo pouca influência no preço final. A análise também revela interdependência entre alguns atributos, como `YearBuilt` e `YearRemodAdd`, o que pode indicar redundância de informação. Esse tipo de visualização é fundamental para orientar a seleção de variáveis em modelos preditivos e entender os fatores mais relevantes para o valor de mercado dos imóveis.


In [ ]:
#Algumas variáveis têm impacto muito maior na previsão que outras.
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor().fit(x_train, y_train)
pd.Series(model.feature_importances_, index=x_train.columns).sort_values().plot(kind='barh')


O gráfico apresenta a importância das variáveis numéricas na predição do preço de venda dos imóveis (`SalePrice`), revelando que a qualidade geral da construção (`OverallQual`) é, de longe, o fator mais influente no modelo, seguida pela área útil acima do solo (`GrLivArea`) e pela metragem do segundo andar (`2ndFlrSF`). Variáveis relacionadas à estrutura física, como área do porão acabado, número de vagas na garagem e ano de construção, também se destacam como elementos relevantes. Em contrapartida, atributos como número de banheiros no porão, valor de itens diversos (`MiscVal`) e área de acabamento de baixa qualidade possuem influência praticamente nula. Essa análise é essencial para identificar quais características devem ser priorizadas em modelos preditivos e reforça que qualidade e espaço são os principais determinantes do valor de mercado de um imóvel.


In [ ]:

# Fazer previsões
y_rl = reg_rl.predict(x_test)
y_ar = reg_arv.predict(x_test)
y_knn = reg_knn.predict(x_test)

# Calcular resíduos
resid_rl = y_test - y_rl
resid_ar = y_test - y_ar
resid_knn = y_test - y_knn

# Plotar resíduos
plt.figure(figsize=(14, 5))
plt.subplot(1, 3, 1)
plt.scatter(y_test, resid_rl, alpha=0.5, color='blue')
plt.axhline(0, color='gray', linestyle='--')
plt.title("Resíduos - Regressão Linear")
plt.xlabel("Valor Real")
plt.ylabel("Resíduo")

plt.subplot(1, 3, 2)
plt.scatter(y_test, resid_ar, alpha=0.5, color='green')
plt.axhline(0, color='gray', linestyle='--')
plt.title("Resíduos - Árvore de Regressão")
plt.xlabel("Valor Real")

plt.subplot(1, 3, 3)
plt.scatter(y_test, resid_knn, alpha=0.5, color='red')
plt.axhline(0, color='gray', linestyle='--')
plt.title("Resíduos - KNN")
plt.xlabel("Valor Real")

plt.tight_layout()
plt.show()


O conjunto de gráficos exibe a distribuição dos resíduos para três modelos de regressão — Linear, Árvore de Regressão e KNN — permitindo avaliar o desempenho preditivo de cada um. Observa-se que a Regressão Linear apresenta um padrão de resíduos em forma de funil, indicando que o modelo tende a errar mais à medida que o valor real dos imóveis aumenta. A Árvore de Regressão, por sua vez, concentra melhor os resíduos em torno de zero, mostrando-se mais flexível, embora ainda apresente outliers. Já o modelo KNN demonstra maior dificuldade com imóveis de alto valor, gerando erros expressivos devido à escassez de vizinhos semelhantes para comparação. Em conjunto, os gráficos sugerem que nenhum dos modelos atinge previsões perfeitamente consistentes e que abordagens mais sofisticadas ou ajustes nos dados podem ser necessários para melhorar a precisão.


## Previsão

In [ ]:
# Selecionar features importantes
features = ['GrLivArea', 'OverallQual', 'YearBuilt', 'FullBath', 'GarageCars', 'Neighborhood']
target = 'SalePrice'
df = df[features + [target]].dropna()

# Separar em X e y
X = df[features]
y = df[target]

# Pré-processamento
categorical = ['Neighborhood']
numerical = ['GrLivArea', 'OverallQual', 'YearBuilt', 'FullBath', 'GarageCars']

preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
], remainder='passthrough')

# Pipeline com regressão linear
model = Pipeline([
    ('preprocess', preprocessor),
    ('regressor', LinearRegression())
])

# Treinar modelo
model.fit(X, y)

# ----------------------------
# Entrada interativa do usuário
# ----------------------------

print("\nDigite os dados da casa para prever o preço:")

# Capturar entradas do usuário
grlivarea = float(input("Área útil da casa (em pés quadrados): "))
overallqual = int(input("Qualidade geral da casa (1 a 10): "))
yearbuilt = int(input("Ano de construção: "))
fullbath = int(input("Número de banheiros completos: "))
garagecars = int(input("Número de vagas de garagem: "))
neighborhood = input("Bairro (ex: CollgCr, NridgHt, OldTown...): ")

# Criar DataFrame com os dados
nova_casa = pd.DataFrame([{
    'GrLivArea': grlivarea,
    'OverallQual': overallqual,
    'YearBuilt': yearbuilt,
    'FullBath': fullbath,
    'GarageCars': garagecars,
    'Neighborhood': neighborhood
}])

# Prever preço
preco_previsto = model.predict(nova_casa)
print(f"\n📈 Preço estimado da casa: ${preco_previsto[0]:,.2f}")
